In [ ]:
!pip install ccxt pandas
!pip install matplotlib

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
import ccxt
import pandas as pd
import time

# 3) Initialize exchange
exchange = ccxt.binance()   # you can swap for 'kraken', 'ftx', etc.

# 4) Parameters
symbol    = 'BTC/USDT'      # crypto pair
timeframe = '1m'            # 1-minute bars
limit     = 500             # max bars per request (exchange-dependent)

# 5) Helper to fetch from `since` until now
def fetch_ohlcv_since(since_ts):
    all_bars = []
    while since_ts < exchange.milliseconds():
        bars = exchange.fetch_ohlcv(symbol, timeframe, since=since_ts, limit=limit)
        if not bars:
            break
        all_bars += bars
        # move since to the last bar's timestamp + 1ms
        since_ts = bars[-1][0] + 1
        time.sleep(exchange.rateLimit / 1000)  # respect rate-limit
    return all_bars

# 6) Choose your start date
start_date = '2025-01-03T00:00:00Z'
since_ts    = exchange.parse8601(start_date)

# 7) Fetch and convert to DataFrame
raw = fetch_ohlcv_since(since_ts)
df  = pd.DataFrame(raw, columns=['timestamp','open','high','low','close','volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# 8) Inspect
print(df.head(), "\n…\n", df.tail())

                         open      high       low     close   volume
timestamp                                                           
2025-01-03 00:00:00  96984.79  97021.19  96984.78  96984.79  8.51875
2025-01-03 00:01:00  96984.78  96984.79  96961.30  96961.31  6.01391
2025-01-03 00:02:00  96961.31  97000.00  96959.81  97000.00  6.38635
2025-01-03 00:03:00  97000.00  97028.08  96999.99  97028.08  4.90563
2025-01-03 00:04:00  97028.07  97028.08  96976.98  96981.56  6.79643 
…
                           open       high        low      close    volume
timestamp                                                                
2025-06-30 11:22:00  107820.00  107849.99  107819.99  107841.91   8.39261
2025-06-30 11:23:00  107841.91  107939.11  107841.90  107910.22  21.52630
2025-06-30 11:24:00  107910.23  107910.23  107872.21  107876.71   3.40893
2025-06-30 11:25:00  107876.71  107876.72  107842.34  107842.35   6.27025
2025-06-30 11:26:00  107842.35  107849.82  107842.34  107849.82   3.0

In [ ]:
!pip install ta

In [3]:
from ta.momentum import RSIIndicator
from ta.trend import EMAIndicator
from ta.volatility import AverageTrueRange

# Compute RSI (14)
rsi = RSIIndicator(close=df['close'], window=14)
df['rsi'] = rsi.rsi()

# Compute EMA200
ema200 = EMAIndicator(close=df['close'], window=200)
df['ema_200'] = ema200.ema_indicator()

# Compute ATR(14)
atr = AverageTrueRange(high=df['high'], low=df['low'], close=df['close'], window=14)
df['atr_14'] = atr.average_true_range()


In [1]:
df.head()

NameError: name 'df' is not defined

In [5]:
df["prev_rsi"] = df["rsi"].shift(1)
breakout_df = df[(df["prev_rsi"] <= 30) & (df["rsi"] > 30)]


In [6]:
df.head()

,open,high,low,close,volume,rsi,ema_200,atr_14,prev_rsi
timestamp,,,,,,,,,
2025-01-03 00:00:00,96984.79,97021.19,96984.78,96984.79,8.51875,NaN,NaN,0.0,NaN
2025-01-03 00:01:00,96984.78,96984.79,96961.30,96961.31,6.01391,NaN,NaN,0.0,NaN
2025-01-03 00:02:00,96961.31,97000.00,96959.81,97000.00,6.38635,NaN,NaN,0.0,NaN
2025-01-03 00:03:00,97000.00,97028.08,96999.99,97028.08,4.90563,NaN,NaN,0.0,NaN
2025-01-03 00:04:00,97028.07,97028.08,96976.98,96981.56,6.79643,NaN,NaN,0.0,NaN


In [7]:
def label_rsi_breakout_profit(df, price_col="close", tp=0.001, sl=-0.001, lookahead=10):
    indices = []
    labels = []
    
    for i in range(1, len(df)):
        # Identify RSI breakout
        if df['rsi'].iloc[i] >= 30 and df['prev_rsi'].iloc[i] < 30:
            entry_price = df[price_col].iloc[i]
            found_label = 0  # default not profitable
            
            for j in range(i + 1, min(i + lookahead, len(df))):
                future_price = df[price_col].iloc[j]
                ret = (future_price - entry_price) / entry_price
                
                if ret >= tp:
                    found_label = 1  # profitable
                    break
                elif ret <= sl:
                    break
            
            indices.append(i)
            labels.append(found_label)
    
    label_df = df.iloc[indices].copy()
    label_df['label'] = labels
    return label_df


In [8]:
temp=label_rsi_breakout_profit(df)
c=0
for i in range(len(temp)):
    if temp['label'][i]==1:
        c+=1
print(c/len(temp))


0.33446519524617996


/var/folders/qd/gbhsmb7j2fq2sjvx6v7yhww80000gn/T/ipykernel_20626/2347605479.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if temp['label'][i]==1:


In [12]:
temp.head()

,open,high,low,close,volume,rsi,ema_200,atr_14,prev_rsi,label
timestamp,,,,,,,,,,
2025-01-03 03:23:00,96870.66,96909.10,96870.65,96909.09,4.00103,38.136114,96988.705864,31.748687,29.750428,0
2025-01-03 04:43:00,96809.53,96863.58,96809.52,96863.58,13.09553,42.074823,96970.333924,27.114738,29.723791,0
2025-01-03 05:09:00,96644.92,96676.36,96644.91,96676.35,4.63782,34.063569,96928.230787,32.310670,27.904131,1
2025-01-03 06:23:00,96624.59,96650.43,96624.59,96650.42,13.03185,33.444003,96840.655091,35.251798,29.343494,0
2025-01-03 07:43:00,96329.92,96414.49,96329.91,96382.00,15.92990,40.919393,96670.663818,42.889210,28.048832,0


In [ ]:
!pip install scikit-learn


In [ ]:
!pip install xgboost

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier

In [14]:
# === Features and target ===
feature_cols = [
    "open", "high", "low", "close", "volume", "rsi", 
    "ema_200", "atr_14", "prev_rsi"
]
X = temp[feature_cols]
y = temp["label"]

# === Train-test split ===
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

# === XGBoost Model ===
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    n_estimators=100,
    max_depth=4,
    learning_rate=0.1,
    random_state=42
)
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)

# === RandomForest Model ===
rf_model = RandomForestClassifier(
    n_estimators=100,
    max_depth=8,
    random_state=42
)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)




/Users/virat/Documents/Trading/.venv/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [17:07:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [15]:
# === Evaluation Function ===
def evaluate(name, y_true, y_pred):
    print(f"==== {name} ====")
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Classification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\n")

# === Evaluate both ===
evaluate("XGBoost", y_test, xgb_pred)
evaluate("Random Forest", y_test, rf_pred)

==== XGBoost ====
Accuracy: 0.6364922206506365
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.86      0.75       454
           1       0.48      0.24      0.32       253

    accuracy                           0.64       707
   macro avg       0.58      0.55      0.54       707
weighted avg       0.60      0.64      0.60       707

Confusion Matrix:
 [[389  65]
 [192  61]]


==== Random Forest ====
Accuracy: 0.6492220650636492
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.88      0.76       454
           1       0.52      0.24      0.33       253

    accuracy                           0.65       707
   macro avg       0.60      0.56      0.55       707
weighted avg       0.62      0.65      0.61       707

Confusion Matrix:
 [[398  56]
 [192  61]]




In [16]:
baseline_winrate = temp["label"].mean() 

In [17]:
baseline_ev = 0.33 * 0.001 + 0.66 * (-0.001)
baseline_ev

-0.00033

In [18]:
model_preds = xgb_model.predict(X_test)
model_probs = xgb_model.predict_proba(X_test)[:, 1]

# Filter predictions
predicted_positive = model_preds == 1
actual_labels = y_test.reset_index(drop=True)

# How many were actually profitable?
true_positive_count = ((actual_labels == 1) & predicted_positive).sum()
false_positive_count = ((actual_labels == 0) & predicted_positive).sum()

total_signaled = predicted_positive.sum()

# Model's predicted winrate
model_winrate = true_positive_count / total_signaled
model_ev = model_winrate * 0.001 + (1 - model_winrate) * (-0.001)
print(f"Model Winrate: {model_winrate:.2%}")
print(f"Model EV: {model_ev:.6f}")

Model Winrate: 48.41%
Model EV: -0.000032


In [19]:
print(true_positive_count, false_positive_count, total_signaled)

61 65 126


In [ ]:
!pip install torch torchvision

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
from tqdm import tqdm

# === Scale features ===
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# === Train/test split ===
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, shuffle=True
)

# === Convert to PyTorch tensors ===
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# === Define ANN ===
class ANN(nn.Module):
    def __init__(self, input_dim):
        super(ANN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# === Initialize model, loss, optimizer ===
model = ANN(X_train.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# === Training loop ===
epochs = 25
for epoch in tqdm(range(epochs),desc="Training"):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

# === Predict ===
model.eval()
with torch.no_grad():
    y_probs = model(X_test_tensor).numpy().flatten()
    y_pred = (y_probs >= 0.5).astype(int)

# === Metrics ===
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# === Calculate Model-Based Profit (Assume TP=0.001, SL=-0.001) ===
tp = 0.001
sl = -0.001
predicted_positive = y_pred == 1
true_positives = ((y_test.values == 1) & predicted_positive).sum()
false_positives = ((y_test.values == 0) & predicted_positive).sum()
total_predicted = predicted_positive.sum()

if total_predicted > 0:
    winrate = true_positives / total_predicted
    expected_profit = winrate * tp + (1 - winrate) * sl
    print(f"Model's expected return per trade: {expected_profit:.6f}")
else:
    print("Model made no positive predictions.")


Training:   0%|          | 0/25 [00:00<?, ?it/s]